# Simple transit model in Sherpa

Mostly to work on issue #59 in the repo, but this should definitley be converted into a tutorial at some point.

Make sure you update your configfile accordingly:

```ini
[setup]
data_set = simple_transit
```

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.constants import G

from sherpa.models import model
from sherpa.data import Data1D
from sherpa.plot import DataPlot
from sherpa.plot import ModelPlot
from sherpa.fit import Fit
from sherpa.stats import LeastSq
from sherpa.optmethods import LevMar
from sherpa.stats import Chi2
from sherpa.plot import FitPlot

os.chdir('../exotic-ism')
import margmodule as marg
from config import CONFIG_INI

## Test 0

Simple dataset with no additional scatter.

In [ ]:
# Test parameters
planet_sys = CONFIG_INI.get('setup', 'data_set')
dtosec = CONFIG_INI.getfloat('constants', 'dtosec')
period = CONFIG_INI.getfloat(planet_sys, 'Per')
Per = period * dtosec
aor = CONFIG_INI.getfloat(planet_sys, 'aor')
constant1 = (G * Per * Per / (4 *np.pi * np.pi))**(1/3)
msmpr = (aor/(constant1))**3
print(msmpr)
print(G.value)
print(Per)

# limb darkening parameters
c1 = 0.0
c2 = 0.0
c3 = 0.0
c4 = 0.0

data_x = np.array([-0.046, -0.044, -0.042, -0.040, -0.038, -0.036, -0.034,
                   -0.032, -0.030, -0.006, -0.004, -0.002, 0.0, 0.002, 0.004,
                   0.006, 0.008, 0.01, 0.032, 0.034, 0.036, 0.038, 0.040,
                   0.042, 0.044, 0.046,0.048])

data_y = np.array([1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.99000000,
                   0.99000000, 0.99000000, 0.99000000, 0.99000000, 0.99000000,
                   0.99000000, 0.99000000, 0.99000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000])

uncertainty = np.array([0.0004] * len(data_x))
sh = np.array([0.] * len(data_x))   # not really needed, but needed for Transit() model setup

In [ ]:
# Quick check of input data
plt.plot(data_x, data_y)
plt.title('Quick test input data')
plt.xlabel('Phase')
plt.ylabel('Flux')

In [ ]:
# Make Sherpa data obkect out of this
data0 = Data1D('example_transit', data_x, data_y, staterror=uncertainty)   # create data object
dplot0 = DataPlot()         # create data *plot* object
dplot0.prepare(data0)   # prepare plot
dplot0.plot()

In [ ]:
# Create and visualize model
model0 = marg.Transit(data_x[0], msmpr, c1, c2, c3, c4, flux0=data_y[0], x_in_phase=True, name='transit', sh=sh)
print(model0)

In [ ]:
# Freeze almost all parameters
model0.flux0.freeze()
model0.epoch.freeze()
model0.inclin.freeze()
model0.msmpr.freeze()
model0.ecc.freeze()
model0.m_fac.freeze()
model0.hstp1.freeze()
model0.hstp2.freeze()
model0.hstp3.freeze()
model0.hstp4.freeze()
model0.xshift1.freeze()
model0.xshift2.freeze()
model0.xshift3.freeze()
model0.xshift4.freeze()

print(model0)

In [ ]:
mplot0 = ModelPlot()
mplot0.prepare(data0, model0)
mplot0.plot()

In [ ]:
# Overplotting current model and data
dplot0.plot()
mplot0.overplot()

In [ ]:
# Set up statistics and optimizer
stat = Chi2()
opt = LevMar()
opt.config['epsfcn'] = np.finfo(float).eps   # adjusting epsfcn to double precision
#print(stat)
print(opt)

# Set up fit
tfit0 = Fit(data0, model0, stat=stat, method=opt)
print('Fit information:')
print(tfit0)

In [ ]:
# Perform the fit
fitresult0 = tfit0.fit()

In [ ]:
print(fitresult0)

### Final results and plot test 0

In [ ]:
# Error from Hessian
calc_errors0 = np.sqrt(fitresult0.extra_output['covar'].diagonal())
rl_err0 = calc_errors0[0]

print('rl = {} +/- {}'.format(model0.rl.val, rl_err0))
print('Reduced chi-squared: {}'.format(fitresult0.rstat))

In [ ]:
mplot0 = ModelPlot()
mplot0.prepare(data0, model0)

dplot0.plot()
mplot0.overplot()

## Test 1

Simple dataset with additional scatter

In [ ]:
random_scatter = np.array([0.32558253, -0.55610514, -1.1150768, -1.2337022, -1.2678875,
                           0.60321692, 1.1025507, 1.5080730, 0.76113001, 0.51978011,
                           0.72241364, -0.086782108, -0.22698337, 0.22780245, 0.47119014,
                           -2.1660677, -1.2477670, 0.28568456, 0.40292731, 0.077955817,
                           -1.1090623, 0.66895172, -0.59215439, 0.79973968, 1.0603756,
                           0.82684954, -1.8334587])
print(random_scatter.shape)

In [ ]:
# Add random scatter
original_y = np.copy(data_y)
data_y = original_y + (random_scatter * uncertainty)

In [ ]:
# Quick loop at scattered data
plt.scatter(data_x, data_y)

In [ ]:
# Make Sherpa data obkect out of this
data1 = Data1D('example_transit', data_x, data_y, staterror=uncertainty)   # create data object
dplot1 = DataPlot()         # create data *plot* object
dplot1.prepare(data1)   # prepare plot
dplot1.plot()

In [ ]:
# Create and visualize model
model1 = marg.Transit(data_x[0], msmpr, c1, c2, c3, c4, flux0=data_y[0], x_in_phase=True, name='transit', sh=sh)
print(model1)

In [ ]:
# Freeze almost all parameters
model1.flux0.freeze()
model1.epoch.freeze()
model1.inclin.freeze()
model1.msmpr.freeze()
model1.ecc.freeze()
model1.m_fac.freeze()
model1.hstp1.freeze()
model1.hstp2.freeze()
model1.hstp3.freeze()
model1.hstp4.freeze()
model1.xshift1.freeze()
model1.xshift2.freeze()
model1.xshift3.freeze()
model1.xshift4.freeze()

print(model1)

In [ ]:
mplot1 = ModelPlot()
mplot1.prepare(data1, model1)
mplot1.plot()

In [ ]:
# Overplotting current model and data
dplot1.plot()
mplot1.overplot()

In [ ]:
# Set up fit
tfit1 = Fit(data1, model1, stat=stat, method=opt)
print('Fit information:')
print(tfit1)

In [ ]:
# Perform the fit
fitresult1 = tfit1.fit()
print(fitresult1)

### Final results and plot test 1

In [ ]:
# Error from Hessian
calc_errors1 = np.sqrt(fitresult1.extra_output['covar'].diagonal())
rl_err1 = calc_errors1[0]

# Results
print('rl = {} +/- {}'.format(model1.rl.val, rl_err1))
print('Reduced chi-squared: {}'.format(fitresult1.rstat))

In [ ]:
mplot1 = ModelPlot()
mplot1.prepare(data1, model1)

dplot1.plot()
mplot1.overplot()

## Test 2

Simple dataset with additional scatter and set limb-darkening coefficients

In [ ]:
# Limb darkening setup
# Values just copied directly from:
# https://github.com/hrwakeford/ExoTiC-ISM/issues/59#issuecomment-533657499

c1 = 0.66396105
c2 = -0.12617095
c3 = 0.053649047
c4 = -0.026713433

# To make sure we use the same numbers, I will just copy the IDL data from here:
# https://github.com/hrwakeford/ExoTiC-ISM/issues/59#issuecomment-533657499
data_y = [1.0001302, 0.99977756, 0.99955397, 0.99950652, 0.99949285, 1.0002413,
          1.0004410, 1.0006032, 1.0003045, 0.98918739, 0.98921560, 0.98886110,
          0.98879472, 0.98898693, 0.98911511, 0.98811305, 0.98855772, 0.98927710,
          1.0001612, 1.0000312, 0.99955638, 1.0002676, 0.99976314, 1.0003199,
          1.0004242, 1.0003307, 0.99926662]

In [ ]:
# Make Sherpa data obkect out of this
data2 = Data1D('example_transit', data_x, data_y, staterror=uncertainty)   # create data object
dplot2 = DataPlot()         # create data *plot* object
dplot2.prepare(data2)   # prepare plot
dplot2.plot()

In [ ]:
# Create and visualize model
model2 = marg.Transit(data_x[0], msmpr, c1, c2, c3, c4, flux0=data_y[0], x_in_phase=True, name='transit', sh=sh)
print(model2)

In [ ]:
# Freeze almost all parameters
model2.flux0.freeze()
model2.epoch.freeze()
model2.inclin.freeze()
model2.msmpr.freeze()
model2.ecc.freeze()
model2.m_fac.freeze()
model2.hstp1.freeze()
model2.hstp2.freeze()
model2.hstp3.freeze()
model2.hstp4.freeze()
model2.xshift1.freeze()
model2.xshift2.freeze()
model2.xshift3.freeze()
model2.xshift4.freeze()

print(model2)

In [ ]:
mplot2 = ModelPlot()
mplot2.prepare(data2, model2)
mplot2.plot()

In [ ]:
# Overplotting current model and data
dplot2.plot()
mplot2.overplot()

In [ ]:
# Set up fit
tfit2 = Fit(data2, model2, stat=stat, method=opt)
print('Fit information:')
print(tfit2)

In [ ]:
# Perform the fit
fitresult2 = tfit2.fit()
print(fitresult2)

### Final results and plot test 2

In [ ]:
# Error from Hessian
calc_errors2 = np.sqrt(fitresult2.extra_output['covar'].diagonal())
rl_err2 = calc_errors2[0]

# Results
print('rl = {} +/- {}'.format(model2.rl.val, rl_err2))
print('Reduced chi-squared: {}'.format(fitresult2.rstat))

In [ ]:
mplot2 = ModelPlot()
mplot2.prepare(data2, model2)

dplot2.plot()
mplot2.overplot()

## Test 3

Simple dataset with additional scatter, set limb-darkening coefficients, and a linear slope

In [ ]:
# Create linear slope
m_fac = 0.04
line = (data_x * m_fac) + 1.00

# Add to y data
#second_y = np.copy(data_y)
#data_y = line * (second_y + (random_scatter*uncertainty))

# To make sure we use the same numbers, I will just copy the IDL data from here:
# https://github.com/hrwakeford/ExoTiC-ISM/issues/59#issuecomment-533665365
data_y = np.array([0.99929017, 0.99901777, 0.99887430, 0.99890683,
                   0.99897318, 0.99980124, 1.0000809, 1.0003231,
                   1.0001044, 0.98993925, 0.99004661, 0.98977091, 0.98978356,
                   0.99005507, 0.99026251, 0.98933832, 0.98986262, 0.99066208, 
                   1.0024415, 1.0023914, 1.0019954, 1.0027883, 1.0023626,
                   1.0030008, 1.0031854, 1.0031717, 1.0021845])

In [ ]:
# Make Sherpa data obkect out of this
data3 = Data1D('example_transit', data_x, data_y, staterror=uncertainty)   # create data object
dplot3 = DataPlot()         # create data *plot* object
dplot3.prepare(data3)   # prepare plot
dplot3.plot()

In [ ]:
# Create and visualize model
model3 = marg.Transit(data_x[0], msmpr, c1, c2, c3, c4, flux0=data_y[0], x_in_phase=True, name='transit', sh=sh)
print(model3)

In [ ]:
# Freeze almost all parameters
# Note how m_fac stays thawed
model3.flux0.freeze()
model3.epoch.freeze()
model3.inclin.freeze()
model3.msmpr.freeze()
model3.ecc.freeze()
model3.hstp1.freeze()
model3.hstp2.freeze()
model3.hstp3.freeze()
model3.hstp4.freeze()
model3.xshift1.freeze()
model3.xshift2.freeze()
model3.xshift3.freeze()
model3.xshift4.freeze()

print(model3)

In [ ]:
# data1 and model3 on purpose, since data didn't change this time
mplot3 = ModelPlot()
mplot3.prepare(data3, model3)
mplot3.plot()

In [ ]:
# Overplotting current model and data
dplot3.plot()
mplot3.overplot()

In [ ]:
# Set up fit
tfit3 = Fit(data3, model3, stat=stat, method=opt)
print('Fit information:')
print(tfit3)

In [ ]:
# Perform the fit
fitresult3 = tfit3.fit()
print(fitresult3)

### Final results and plot test 3

In [ ]:
# Errors from Hessian
calc_errors3 = np.sqrt(fitresult3.extra_output['covar'].diagonal())
rl_err3 = calc_errors3[0]
m_fac_err3 = calc_errors3[1]

# Results
print('rl = {} +/- {}'.format(model3.rl.val, rl_err3))
print('m_fac = {} +/- {}'.format(model3.m_fac.val, m_fac_err3))
print('Reduced chi-squared: {}'.format(fitresult3.rstat))

In [ ]:
mplot3 = ModelPlot()
mplot3.prepare(data3, model3)

dplot3.plot()
mplot3.overplot()